In [ ]:
# Install everything in one go
%pip install --quiet requests>=2.28.0 beautifulsoup4>=4.11.0 PyPDF2>=3.0.0 python-dotenv

import os
import requests
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
from dotenv import load_dotenv

# Load .env
load_dotenv()

# Config vars
API_KEY       = os.getenv("API_KEY")
API_URL_BASE  = os.getenv("API_URL")
PROMPT_FILE   = os.getenv("PROMPT_FILE")
RAW_TEXT_FILE = os.getenv("RAW_TEXT_FILE")
WEBPAGE_URL   = os.getenv("WEBPAGE_URL")
PDF_FILE      = os.getenv("PDF_FILE")
OUTPUT_FILE   = os.getenv("OUTPUT_FILE")
TEMPERATURE   = float(os.getenv("TEMPERATURE", 0.0))
API_URL       = f"{API_URL_BASE}?key={API_KEY}"

def read_file(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        raise RuntimeError(f"File not found: {path}")

def fetch_webpage_text(url: str) -> str:
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()
    return soup.get_text(separator="\n").strip()

def extract_pdf_text(path: str) -> str:
    try:
        reader = PdfReader(path)
    except FileNotFoundError:
        raise RuntimeError(f"PDF not found: {path}")
    text_pages = []
    for page in reader.pages:
        page_text = page.extract_text() or ""
        text_pages.append(page_text)
    return "\n".join(text_pages)

def generate_content(prompt_text: str, temperature: float) -> dict:
    headers = {"Content-Type": "application/json"}
    body = {
        "contents": [{"parts": [{"text": prompt_text}]}],
        "generationConfig": {"temperature": temperature},
    }
    resp = requests.post(API_URL, headers=headers, json=body)
    resp.raise_for_status()
    return resp.json()

# —— Run all steps in this cell —— 

# 1. Read sources
prompt_with_examples = read_file(PROMPT_FILE)
raw_text             = read_file(RAW_TEXT_FILE)

# 2. Build final prompt
final_prompt = (
    f"{prompt_with_examples}\n\n"
    f"Raw Text to Anonymize:\n{raw_text}\n\n"
)

# 3. Call API
output = generate_content(final_prompt, TEMPERATURE)

# 4. Write & print response
try:
    response_text = output["candidates"][0]["content"]["parts"][0]["text"]
    with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
        f.write(response_text)
    print(response_text)
except (KeyError, IndexError):
    print("Unexpected API response format:")
    print(output)
